<a href="https://colab.research.google.com/github/gamecicn/TF2_Study/blob/KaggleTwitterDiaster/T2E2_DiasterTwitter_LSTM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Score : 0.78363
# Rank : 889/1216

In [2]:
import numpy as np
import nltk
import tensorflow as tf
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [3]:
df_data = pd.read_csv('/content/drive/MyDrive/DS_data/TwitterDisaster/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/DS_data/TwitterDisaster/test.csv')

In [4]:
df_data_copy = df_data.copy()

df_train  = df_data_copy.sample(frac=0.99, random_state=0)
df_valid = df_data_copy.drop(df_train.index)

df_train_labels = df_train.pop('target')
df_valid_labels = df_valid.pop('target')

print("lenght of train is {}".format(len(df_train)))
print("lenght of valid is {}".format(len(df_valid)))

lenght of train is 7537
lenght of valid is 76


In [5]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped  = tf.strings.regex_replace(lowercase, '<br />', ' ')
  stripped  = tf.strings.regex_replace(lowercase, '#', ' ')
  stripped  = tf.strings.regex_replace(lowercase, '@', ' ')
  return stripped

## Vectorizlize Text

In [16]:
VOCAB_SIZE = 10000
sequence_length = 140

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (without labels), then call adapt
vectorize_layer.adapt(df_train['text'].to_numpy())

### Build  Model

In [17]:

model = tf.keras.Sequential([
  vectorize_layer,                           
  layers.Embedding(VOCAB_SIZE , 32),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1)
  ])



In [18]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics='accuracy')

epochs = 10
history = model.fit(
    df_train['text'].to_numpy(), 
    df_train_labels,
    epochs=epochs)

model.summary()

Epoch 1/10
236/236 [==============================] - 7s 17ms/step - loss: 0.6883 - accuracy: 0.5719
Epoch 2/10
236/236 [==============================] - 4s 16ms/step - loss: 0.6608 - accuracy: 0.5708
Epoch 3/10
236/236 [==============================] - 4s 17ms/step - loss: 0.5383 - accuracy: 0.6935
Epoch 4/10
236/236 [==============================] - 4s 16ms/step - loss: 0.4275 - accuracy: 0.7952
Epoch 5/10
236/236 [==============================] - 4s 17ms/step - loss: 0.3668 - accuracy: 0.8307
Epoch 6/10
236/236 [==============================] - 4s 17ms/step - loss: 0.3287 - accuracy: 0.8542
Epoch 7/10
236/236 [==============================] - 4s 17ms/step - loss: 0.2832 - accuracy: 0.8770
Epoch 8/10
236/236 [==============================] - 4s 16ms/step - loss: 0.2505 - accuracy: 0.8904
Epoch 9/10
236/236 [==============================] - 4s 16ms/step - loss: 0.2278 - accuracy: 0.9030
Epoch 10/10
236/236 [==============================] - 4s 17ms/step - loss: 0.1984 - accura

## Evaluate the model

In [19]:
loss, accuracy = model.evaluate(df_valid['text'], df_valid_labels)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

3/3 [==============================] - 1s 11ms/step - loss: 0.5533 - accuracy: 0.7500
Loss:  0.553347647190094
Accuracy:  0.75


## Prediction

In [20]:
prediction_model = tf.keras.Sequential([model, 
                                        layers.Activation('sigmoid')])

predictions = prediction_model.predict(df_test['text'].to_numpy())



In [25]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [27]:
res = pd.DataFrame(data = {"id" : df_test['id'] , 
                           "target"   : map(lambda x: '0' if x<0.5 else '1', predictions)})

res.to_csv("/content/drive/MyDrive/output/Kaggle_twitterDisaster_LSTM1.csv", index=False)